## Word Embedding Experiments with the Wikipedia Corpus using PMI Embeddings 

### Pre-processing 

In [3]:
import nltk
from numpy.linalg import inv
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/varshant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/varshant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
f = open("/project/cmsc25025/wikipedia/wiki-text.txt", "r")
wiki_contents = f.read()
stop_words = set(stopwords.words('english'))

In [5]:
corpus = wiki_contents.split(' ')
words = list(corpus)

In [6]:
words = [w for w in words if w not in stop_words]

In [7]:
fdist = nltk.FreqDist(words)

In [8]:
vocab_word = sorted(w for w in set(words) if fdist[w] > 500)

In [9]:
index_dict = {};
for idx, word in enumerate(vocab_word):
    index_dict[word] = idx
vocab_len = len(vocab_word)
vocab_word = set(vocab_word)

In [10]:
corpus = [' ', ' ', ' ', ' ', ' '] + corpus
corpus = corpus + [' ', ' ', ' ', ' ', ' ']

In [11]:
context_word_arr = np.zeros((vocab_len, vocab_len));

### Compute PMI 

In [12]:
for indx, word in enumerate(words):
    if word in vocab_word:
        i = index_dict[word]
        for k in range(1,5):
            if words[indx - k] in vocab_word:
                j = index_dict[words[indx - k]]
                context_word_arr[i, j] += 1
            if words[indx + k] in vocab_word:
                j = index_dict[words[indx + k]]
                context_word_arr[i, j] += 1
    if (indx % 10000 == 0):
        print indx
        

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

10370000
10380000
10390000
10400000
10410000
10420000
10430000
10440000
10450000
10460000
10470000
10480000
10490000
10500000
10510000
10520000
10530000
10540000
10550000
10560000
10570000
10580000
10590000
10600000
10610000
10620000
10630000
10640000
10650000
10660000
10670000
10680000
10690000
10700000
10710000
10720000
10730000
10740000
10750000
10760000
10770000
10780000
10790000
10800000
10810000
10820000
10830000
10840000
10850000
10860000
10870000
10880000
10890000
10900000
10910000
10920000
10930000
10940000
10950000
10960000
10970000
10980000
10990000
11000000
11010000
11020000
11030000
11040000
11050000
11060000
11070000
11080000
11090000
11100000
11110000
11120000
11130000
11140000
11150000
11160000
11170000
11180000
11190000
11200000
11210000
11220000
11230000
11240000
11250000
11260000
11270000
11280000
11290000
11300000
11310000
11320000
11330000
11340000
11350000
11360000
11370000
11380000
11390000
11400000
11410000
11420000
11430000
11440000
11450000
11460000
11470000
1

19490000
19500000
19510000
19520000
19530000
19540000
19550000
19560000
19570000
19580000
19590000
19600000
19610000
19620000
19630000
19640000
19650000
19660000
19670000
19680000
19690000
19700000
19710000
19720000
19730000
19740000
19750000
19760000
19770000
19780000
19790000
19800000
19810000
19820000
19830000
19840000
19850000
19860000
19870000
19880000
19890000
19900000
19910000
19920000
19930000
19940000
19950000
19960000
19970000
19980000
19990000
20000000
20010000
20020000
20030000
20040000
20050000
20060000
20070000
20080000
20090000
20100000
20110000
20120000
20130000
20140000
20150000
20160000
20170000
20180000
20190000
20200000
20210000
20220000
20230000
20240000
20250000
20260000
20270000
20280000
20290000
20300000
20310000
20320000
20330000
20340000
20350000
20360000
20370000
20380000
20390000
20400000
20410000
20420000
20430000
20440000
20450000
20460000
20470000
20480000
20490000
20500000
20510000
20520000
20530000
20540000
20550000
20560000
20570000
20580000
20590000
2

28620000
28630000
28640000
28650000
28660000
28670000
28680000
28690000
28700000
28710000
28720000
28730000
28740000
28750000
28760000
28770000
28780000
28790000
28800000
28810000
28820000
28830000
28840000
28850000
28860000
28870000
28880000
28890000
28900000
28910000
28920000
28930000
28940000
28950000
28960000
28970000
28980000
28990000
29000000
29010000
29020000
29030000
29040000
29050000
29060000
29070000
29080000
29090000
29100000
29110000
29120000
29130000
29140000
29150000
29160000
29170000
29180000
29190000
29200000
29210000
29220000
29230000
29240000
29250000
29260000
29270000
29280000
29290000
29300000
29310000
29320000
29330000
29340000
29350000
29360000
29370000
29380000
29390000
29400000
29410000
29420000
29430000
29440000
29450000
29460000
29470000
29480000
29490000
29500000
29510000
29520000
29530000
29540000
29550000
29560000
29570000
29580000
29590000
29600000
29610000
29620000
29630000
29640000
29650000
29660000
29670000
29680000
29690000
29700000
29710000
29720000
2

37760000
37770000
37780000
37790000
37800000
37810000
37820000
37830000
37840000
37850000
37860000
37870000
37880000
37890000
37900000
37910000
37920000
37930000
37940000
37950000
37960000
37970000
37980000
37990000
38000000
38010000
38020000
38030000
38040000
38050000
38060000
38070000
38080000
38090000
38100000
38110000
38120000
38130000
38140000
38150000
38160000
38170000
38180000
38190000
38200000
38210000
38220000
38230000
38240000
38250000
38260000
38270000
38280000
38290000
38300000
38310000
38320000
38330000
38340000
38350000
38360000
38370000
38380000
38390000
38400000
38410000
38420000
38430000
38440000
38450000
38460000
38470000
38480000
38490000
38500000
38510000
38520000
38530000
38540000
38550000
38560000
38570000
38580000
38590000
38600000
38610000
38620000
38630000
38640000
38650000
38660000
38670000
38680000
38690000
38700000
38710000
38720000
38730000
38740000
38750000
38760000
38770000
38780000
38790000
38800000
38810000
38820000
38830000
38840000
38850000
38860000
3

46890000
46900000
46910000
46920000
46930000
46940000
46950000
46960000
46970000
46980000
46990000
47000000
47010000
47020000
47030000
47040000
47050000
47060000
47070000
47080000
47090000
47100000
47110000
47120000
47130000
47140000
47150000
47160000
47170000
47180000
47190000
47200000
47210000
47220000
47230000
47240000
47250000
47260000
47270000
47280000
47290000
47300000
47310000
47320000
47330000
47340000
47350000
47360000
47370000
47380000
47390000
47400000
47410000
47420000
47430000
47440000
47450000
47460000
47470000
47480000
47490000
47500000
47510000
47520000
47530000
47540000
47550000
47560000
47570000
47580000
47590000
47600000
47610000
47620000
47630000
47640000
47650000
47660000
47670000
47680000
47690000
47700000
47710000
47720000
47730000
47740000
47750000
47760000
47770000
47780000
47790000
47800000
47810000
47820000
47830000
47840000
47850000
47860000
47870000
47880000
47890000
47900000
47910000
47920000
47930000
47940000
47950000
47960000
47970000
47980000
47990000
4

65100000
65110000
65120000
65130000
65140000
65150000
65160000
65170000
65180000
65190000
65200000
65210000
65220000
65230000
65240000
65250000
65260000
65270000
65280000
65290000
65300000
65310000
65320000
65330000
65340000
65350000
65360000
65370000
65380000
65390000
65400000
65410000
65420000
65430000
65440000
65450000
65460000
65470000
65480000
65490000
65500000
65510000
65520000
65530000
65540000
65550000
65560000
65570000
65580000
65590000
65600000
65610000
65620000
65630000
65640000
65650000
65660000
65670000
65680000
65690000
65700000
65710000
65720000
65730000
65740000
65750000
65760000
65770000
65780000
65790000
65800000
65810000
65820000
65830000
65840000
65850000
65860000
65870000
65880000
65890000
65900000
65910000
65920000
65930000
65940000
65950000
65960000
65970000
65980000
65990000
66000000
66010000
66020000
66030000
66040000
66050000
66060000
66070000
66080000
66090000
66100000
66110000
66120000
66130000
66140000
66150000
66160000
66170000
66180000
66190000
66200000
6

74240000
74250000
74260000
74270000
74280000
74290000
74300000
74310000
74320000
74330000
74340000
74350000
74360000
74370000
74380000
74390000
74400000
74410000
74420000
74430000
74440000
74450000
74460000
74470000
74480000
74490000
74500000
74510000
74520000
74530000
74540000
74550000
74560000
74570000
74580000
74590000
74600000
74610000
74620000
74630000
74640000
74650000
74660000
74670000
74680000
74690000
74700000
74710000
74720000
74730000
74740000
74750000
74760000
74770000
74780000
74790000
74800000
74810000
74820000
74830000
74840000
74850000
74860000
74870000
74880000
74890000
74900000
74910000
74920000
74930000
74940000
74950000
74960000
74970000
74980000
74990000
75000000
75010000
75020000
75030000
75040000
75050000
75060000
75070000
75080000
75090000
75100000
75110000
75120000
75130000
75140000
75150000
75160000
75170000
75180000
75190000
75200000
75210000
75220000
75230000
75240000
75250000
75260000
75270000
75280000
75290000
75300000
75310000
75320000
75330000
75340000
7

IndexError: list index out of range

In [13]:
num_pairs = np.count_nonzero(context_word_arr)
totals = np.sum(context_word_arr, axis=1)

vocab_len = len(vocab_word)
M = np.zeros((vocab_len, vocab_len))
for index, x in np.ndenumerate(M):
    i = index[0]
    j = index[1]
    M[i, j] = np.log(((context_word_arr[i, j] + 1) * num_pairs ) / (totals[i] * totals[j]))
        

### k-SVD of PMI Matrix 

In [14]:
import scipy
U, s, V = scipy.sparse.linalg.svds(scipy.sparse.csr_matrix(M), k=50)

In [15]:
s = np.diag(s)
W = np.dot(U, np.sqrt(s))
import pickle 
np.save('embeddings', W)
W.shape

(13201, 50)

### Find similarities 

In [16]:
index_word = {}
for key in index_dict.keys():
    index_word[index_dict[key]] = key

In [17]:
def get_similar(word, n=5):
    similarities = np.dot(U, U[index_dict[word]])
    most_similar = similarities.argsort()
    most_similar = most_similar[::-1]
    for i in range(n):
        print(index_word[most_similar[i]])

In [18]:
get_similar('physics')

physics
science
mathematics
university
theory


In [19]:
get_similar('republican')

republican
election
party
senator
democratic


In [20]:
get_similar('einstein')

physics
theory
quantum
mathematics
mathematical


In [21]:
get_similar('algebra')

theorem
algebra
frac
vector
finite


In [22]:
get_similar('fish')

species
food
fish
fruit
birds


### Finding linear analogies

In [23]:
def analogies(w1, w2, w3):
    v1 = U[index_dict[w1]]
    v2 = U[index_dict[w2]]
    v3 = U[index_dict[w3]]
    v = v1 - v2 + v3
    most_similar = np.dot(U, v)
    most_similar = most_similar.argsort()
    most_similar = most_similar[::-1]
    
    for i in range(5):
        print(index_word[most_similar[i]])

In [24]:
print("france : paris :: england : london")
analogies('paris', 'france', 'england' )

france : paris :: england : london
england
london
scotland
wales
st


In [25]:
print("republican : democrat :: conservative : liberal")
analogies('republican', 'democrat', 'conservative' )

republican : democrat :: conservative : liberal
party
conservative
liberal
election
republican


In [26]:
print("thief : bad :: child : daughter")
analogies('thief', 'bad', 'child' )

thief : bad :: child : daughter
daughter
mother
child
wife
marriage


In [27]:
print("nearby : remote :: native : language")
analogies('remote', 'nearby', 'native' )

nearby : remote :: native : language
native
language
languages
african
people


In [28]:
print("happy : smile :: pain : disease")
analogies('happy', 'smile', 'pain' )

happy : smile :: pain : disease
disease
symptoms
pain
patients
heart


In [29]:
print("sad : anger :: happy : love")
analogies('sad', 'anger', 'happy' )

sad : anger :: happy : love
want
happy
love
girl
song
